In [1]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle


from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim


[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# def space_punc(sent):
#     new_sent=sent
#     add=0
#     for m in re.finditer(r'[\d\-\/\"\';,.!?\)\(]+',sent):
#         if m.end(0)+add>=len(new_sent):
#             return new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]
#         else:
#             new_sent=new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]+' '+new_sent[m.end(0)+add:]
#             add+=2
#     return new_sent
        
# def mult_sim(w, target_v, context_v):
#     target_similarity = w.dot(target_v)
#     target_similarity[target_similarity<0] = 0.0
#     context_similarity = w.dot(context_v)
#     context_similarity[context_similarity<0] = 0.0
#     return (target_similarity * context_similarity)


# def find_unk_pos(words,test_w):
#     try:
#         pos=words.index(test_w.upper())
#     except ValueError:
#         try:
#             pos=words.index(test_w.upper()+'S')
#         except ValueError as e:
#             try:
#                 pos=words.index(test_w.upper()+'ES')
#             except ValueError as e:
#                 try:
#                     pos=words.index(test_w.upper()[:-1]+'IES')
#                 except ValueError as e:
#                     try:
#                         pos=words.index(test_w.upper()+test_w.upper()[-1]+'ES')
#                     except ValueError as e:
#                         print (e)
#     return pos

def produce_top_n_simwords(w_filter,context_embed,n_result,index2word):
        # compute top n_result similarity weights

        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    print('{0}: {1}'.format(str(index2word[i]), str(similarity[i])))
                    count += 1
                    top_words_i.append(i)
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        return top_vec
def skipgram_context(model,words,pos):
    context_wvs=[]
    for i,word in enumerate(words):
        if i != pos:
            try:
                context_wvs.append(model[word])
                
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    context_embed=sum(np.array(context_wvs))/len(context_wvs)
    return len(context_wvs),context_embed
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight):
    test_s=test_s.replace(test_w, ' '+test_w+' ')

    #test_s=space_punc(test_s)
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    #decide on model
    if model_type=='context2vec':
        
        context_embed= model.context2vec(words, pos)
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
    
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos)
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
        
    #decide on weight 
    if weight=='top_mutual_sim':
        top_vec=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        score=sum(sum(top_vec.dot(top_vec.T)))/(n_result**2)
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return score,context_embed

def additive_model(test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False):
    context_out=[]
    context_weights=[]
    for test_s in test_ss.split('@@'):
        test_s=test_s.strip()
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight)
        print ('weight is {0}'.format(score))
        context_out.append(context_embed)
        context_weights.append(score)
    context_out=np.array(context_out)
    norm_weights=np.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
    print ('normalized weight: \n  {0}'.format(norm_weights))
    context_avg=sum(norm_weights*context_out)
    print('producing top {0} words for new embedding'.format(n_result))
    top_vec=produce_top_n_simwords(w_filter,context_avg,n_result,index2word)
     
    return context_avg

def filter_w(w,word2index,index2word,word_freq_f):
    #filter out words with freq less than 200, words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    with open(word_freq_f) as f:
        for line in f:
            f_w=line.split()
            if len(f_w)>1:
                if f_w[1] in word2index and re.search('[a-zA-Z]',f_w[1])!=None and f_w[1] not in stopw :
                    #word2freq_nostop[f_w[1]]=f_w[0]
                    index_filter2index.append(word2index[f_w[1]])
                    word2index_filter[f_w[1]]=counter
                    index2word_filter[counter]=f_w[1]
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model


def eval_chimera(chimeras_data_dir,num_sent,context_model,model_type,n_result,w,index2word,weight=False):
    golds=[]
    model_predict=[]
    num_indic='l'+str(num_sent)
    for root, subdir, fname in os.walk(chimeras_data_dir):
        for fn in fname:
            if fn.endswith('fixed.test.txt')and 'l'+str(num_sent)==fn.split('.')[1]: #read in the test file
                print (fn)
                data=pd.read_csv(os.path.join(chimeras_data_dir,fn),delimiter='\t')
                for index, row in data.iterrows():
                    
                    #compute context representation
                    if weight!='learned':
                        context_avg=additive_model(row[1],'___', model_type,context_model,n_result,w,index2word,weight)
                   
                    
                    #cosine similarity with probe embedding
                    for gold,probe in zip(row[3].split(','),row[2].split(',')):
                        try:
                            cos=w[word2index[probe]].dot(context_avg)
                            if xp.isnan(cos):
                                continue
                            else:
                                model_predict.append(cos)
                                golds.append(gold)
                        except KeyError as e:
                            print ("====warning key error for probe=====: {0}".format(e))
    if len(golds)==len(model_predict):
        print ('spearman correlation is {0}'.format(spearmanr(golds,model_predict)))
    else:
        print ('unequal length: gold {0}, model {1}'.format( len(golds),len(model_predict)))


    

{'tripolitan': 151704,
 'biennials': 121973,
 'tsukino': 186810,
 'biysk': 256933,
 'woode': 193350,
 'verplank': 206266,
 'nunnery': 31611,
 'woodi': 253296,
 'gavar': 248756,
 'sowell': 65731,
 'utnapishtim': 169059,
 'fleeces': 139027,
 'woods': 3881,
 'clotted': 119004,
 'spiders': 11318,
 'gavan': 76144,
 'woody': 8886,
 'trawling': 52416,
 'comically': 43315,
 'gae': 117687,
 'spidery': 243024,
 'regularize': 138926,
 'caner': 188261,
 'yanagisawa': 133074,
 'canet': 99640,
 'fibrillose': 183122,
 'brodmann-1909': 227496,
 'sprague': 23072,
 'caney': 58478,
 'jairam': 132359,
 'kurigalzu': 241424,
 'pager': 65539,
 'yusaf': 235790,
 'chanthaburi': 143868,
 'igual': 177753,
 'fsos': 135740,
 'caned': 108096,
 'mirisch': 115073,
 'clottey': 170473,
 'rickman': 41966,
 'canek': 146088,
 'canem': 258149,
 'afikoman': 240201,
 'heliothis': 144725,
 'slothful': 218480,
 'naturopathic': 81903,
 'sidestrand': 232897,
 'grenadiers': 33291,
 'trojan': 15353,
 'pigment': 16806,
 'caryopsis'

In [29]:
# model = gensim.models.Word2Vec.load('../models/wiki_all.model/wiki_all.sent.split.model')
# stopw=stopwords.words('english')
# stopw=[word.encode('utf-8') for word in stopw]
# model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
# model['it']

KeyError: 'it'

In [29]:

if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
        model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
        model_type='context2vec'
        context_rm_stopw=0

#         model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
#         model_type='skipgram'
#         context_rm_stopw=1

        weight='top_mutual_sim'
        data='./eval_data/data-chimeras'
    else:
        if len(sys.argv) < 3:
            print >> sys.stderr, "Usage: %s <model_param_file> <model_type> <weight> <context_rm_stop>"  % (sys.argv[0])
            sys.exit(1)

        model_param_file = sys.argv[1]
        model_type=sys.argv[2]
        weight=int(sys.argv[3])
        context_rm_stop=int.argv[4]
        data =argv[5]
    
    
    #gpu setup 
    gpu = -1 # todo: make this work with gpu

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
    elif model_type=='skipgram':
        model = gensim.models.Word2Vec.load(model_param_file)
        w=model.wv.vectors
        #vector normalize for probe w embedding
        s = np.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model.wv.index2word
        word2index={key: model.wv.vocab[key].index for key in model.wv.vocab}
        
        
    #weight
    if weight=='top_mutual_sim':
        #filter w : remove stop words and low freq words 
        print ('filter words for target....')
        w,word2index,index2word=filter_w(w,word2index,index2word,'word_freq')
        n_result = 20
        
    # remove context stop words
    if int(context_rm_stopw)==1:
        print ('filter words for context....')

        model=rm_stopw_context(model)
        
    

read model....
Reading config file: ../models/context2vec/model_dir/context2vec.ukwac.model.params
Config:  {'config_path': '../models/context2vec/model_dir/', 'model_file': 'context2vec.ukwac.model', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'context2vec.ukwac.words.targets', 'unit': '300'}
filter words for target....


In [30]:
#read in data
if data.split('/')[-1]== 'data-chimeras':
        eval_chimera(data,2,model,model_type,n_result,w,index2word,weight)

dataset.l2.fixed.train.txt
the book is dated 1707 and contains numerous recipes for vegetables including one  ___ 
producing top 20 simwords
recipe: 0.49666485
truffle: 0.49283758
dish: 0.48631617
bottle: 0.4858039
piece: 0.4848786
staple: 0.48471606
leaf: 0.4838686
goose: 0.48315984
rum: 0.4830834
unmounted: 0.4825816
cookbook: 0.48249382
girdle: 0.4811394
envelope: 0.48097542
lamb: 0.48091343
kippers: 0.48063692
haggard: 0.4805138
pbk: 0.4803611
dairy: 0.4802178
parcel: 0.47985527
veal: 0.479479
weight is 0.258999906182
add the carrots  ___  and water and cook gently until the vegetables are tender
producing top 20 simwords
fennel: 0.54035234
tarragon: 0.53774375
mixture: 0.5337665
2tbsp: 0.53191394
eggs: 0.53083557
gently: 0.52945143
lengthways: 0.5288544
prunes: 0.52877855
1tsp: 0.5283899
parsley: 0.52836263
cornflour: 0.5277069
salt: 0.52519274
suet: 0.52515453
marinade: 0.52510935
casserole: 0.52499145
buttermilk: 0.52496284
lightly: 0.5248032
sparingly: 0.52447945
1tbsp: 0.52446

parsnip: 0.5417523879374977
lettuce: 0.5343930746672707
carrot: 0.5339812705446525
turnip: 0.5334835136810769
leek: 0.5332827564865145
courgette: 0.5330256873388425
lemon: 0.532917433597945
spud: 0.5328059014703469
cabbage: 0.5322034840264789
tarragon: 0.5313374028456459
tomato: 0.5308348950655752
fennel: 0.530745531898277
cucumber: 0.5297095469429322
swede: 0.5295730423344497
turbot: 0.5289717295429675
lamb: 0.5283252437022622
chicken: 0.5281418199969645
haddock: 0.5279469986712519
moose: 0.527763951463652
lentil: 0.5272776127395231
in autumn and winter the cargoes were damsons tomatoes marrows  ___  apples and pears
producing top 20 simwords
pulped: 0.5268594
roasted: 0.5252885
ripening: 0.52285415
baked: 0.52233267
candied: 0.5187237
blanched: 0.518282
frying: 0.51435596
selling: 0.5142253
ripened: 0.51203406
bought: 0.5099683
boiled: 0.50915384
cooked: 0.5090486
sliced: 0.5090057
harvesting: 0.50855786
pickling: 0.5085042
malted: 0.5078079
creamed: 0.50760335
fom: 0.5074231
weighin

blender: 0.5180747156268063
casserole: 0.5153394235684441
wok: 0.5144182762769667
dish: 0.5133449603931811
cake: 0.5123265488496095
cream: 0.5118508145716755
saucepan: 0.5116743492788235
sponge: 0.509858337658741
pan: 0.5098004613201924
barbecue: 0.5097702805515502
foil: 0.5097667679863219
sieve: 0.5094591765136246
jug: 0.5092656261383958
meringue: 0.5089621430108243
sauce: 0.5085124593500907
margarine: 0.5084959106008408
batter: 0.5083508030029119
trim: 0.508162762790936
fridge: 0.5079352725870983
skillet: 0.5076742758121463
====warning key error for probe=====: 'wastebin'
we ate in the hotel and had an interesting chat with a travelling salesman from milan who  ___ 
producing top 20 simwords
telephoned: 0.49386883
wrote: 0.49370548
semi-retired: 0.4928025
shopped: 0.4924544
booked: 0.4916812
compered: 0.49149162
lived: 0.49147558
visited: 0.4904978
sold: 0.4891342
insisted: 0.4889955
knows: 0.4889626
qualified: 0.4882286
phoned: 0.48708707
includes:-: 0.48691896
tasted: 0.48667678
te

chiller: 0.5157053972487277
hotplate: 0.5156184498604264
machine: 0.515504295064044
oven: 0.5149230799742202
extinguisher: 0.5131862233748766
appliance: 0.5128538112021231
foil: 0.5109753340486366
evaporator: 0.5099506976658781
stirrer: 0.5092916683299381
grill: 0.5091590478932984
hob: 0.5088576267619547
cartridge: 0.508744126091963
jug: 0.5084174613238195
kettle: 0.5080274503477756
thermometer: 0.5079512539282125
carton: 0.5076401659670723
container: 0.5070573127010795
wok: 0.5067724192666835
cooler: 0.5065531223997424
stove: 0.5064502218341813
====warning key error for probe=====: 'wastebin'
leading this exciting young band is seb rochford a man who lets his compositions and his  ___  do the talking
producing top 20 simwords
musicians: 0.55384135
songs: 0.54990554
music: 0.5489115
bandmates: 0.5454433
talent: 0.54536176
performances: 0.54521
sidekicks: 0.5446004
vocals: 0.5444851
band: 0.54398775
accordion: 0.54322433
lyrics: 0.54309
contemporaries: 0.5412601
colleagues: 0.54112315
o

gallant: 0.5221340271069543
life-affirming: 0.5211179327611739
maiden: 0.5208181762717075
sudden: 0.5189884424679194
fist: 0.5185362298925316
heroic: 0.5165725171640647
brave: 0.5150671049690894
thunderous: 0.5143914809535044
whole-hearted: 0.5128898488560749
courageous: 0.5123852427050706
fearless: 0.5121049645433873
victorious: 0.5114927306115808
fateful: 0.5114313060427834
triumphant: 0.510883104687995
glorious: 0.5107341286204404
heartless: 0.5098685577325578
bold: 0.5096196576717563
swift: 0.509019284903596
mighty: 0.5089392998561404
grim: 0.5087272311859775
how fine your life used to be runninaround bangingyour  ___  like it s
producing top 20 simwords
sounds: 0.55438983
sounded: 0.54072446
kinda: 0.5370424
looks: 0.5359881
jist: 0.5358089
sorta: 0.5343471
would'nt: 0.5329245
wud: 0.5305825
don?t: 0.53036976
somthing: 0.53032404
.i: 0.5300488
summat: 0.5297795
didnt: 0.5286795
sounding: 0.52706426
sound: 0.525961
feels: 0.5254882
felt: 0.5244997
'd: 0.5236764
dont: 0.5227693
jus:

...

harp: 0.5219319541280799
flute: 0.5206258973042689
chorus: 0.5202395314994924
edge.: 0.517324182145357
chorale: 0.5163004206403331
treble: 0.5161206516273259
melody: 0.514342739021323
quavers: 0.5140711539808488
bass: 0.5132594656372841
drum: 0.5129359992452894
coda: 0.5118243805265253
motet: 0.5116189140551123
diapason: 0.5112913260933741
trill: 0.5101716151676976
choir: 0.5101108044347592
chanter: 0.5098529394413198
chords: 0.5098164603567402
horn: 0.5093592918010035
pedal: 0.5084597538434223
fanfares: 0.5084381595472324
millie is setting up two music stands and lugging her  ___  case in from the hall
producing top 20 simwords
pillow: 0.5392743
cigarette: 0.5324642
cardboard: 0.53214854
mirror: 0.52733564
satchel: 0.52708393
cello: 0.52593404
styrofoam: 0.52556086
guitar: 0.52469915
tinny: 0.5240373
freezer: 0.52219725
top: 0.5221877
half-finished: 0.5221692
glass: 0.52181834
violin: 0.5217752
hand.: 0.5210887
briefcase: 0.5210383
pyjama: 0.520767
prized: 0.52069193
basket: 0.5203585

anatomy: 0.5147710108247714
exam: 0.5082783170985434
handwriting: 0.507444823275206
tongue: 0.5067108605689535
chart: 0.5058557248550206
manuscript: 0.5052786591587648
recipe: 0.5043957524060004
book: 0.5039751741543324
routine: 0.5030146766377617
dictionary: 0.5029287032183062
histology: 0.5025494393489314
artefact: 0.5022406928464398
keyboard: 0.501960332969281
paper: 0.5018540356650063
thesis: 0.5018158752800356
organ: 0.5017393026981212
skeleton: 0.5015796983576475
tape: 0.5013865760013603
textbook: 0.5011496095975886
calculus: 0.5010481152931514
she remembered the sketches of mice and the dusty  ___  and blushed at the recollection
producing top 20 simwords
drawers: 0.5394396
skulls: 0.5375138
ghosts: 0.53590447
skull: 0.5349364
arm-chair: 0.53457075
flagstones: 0.53444695
cobwebs: 0.53313345
lids: 0.53266877
eyes: 0.5326288
dustbins: 0.5325073
bones: 0.5323815
mummies: 0.5323033
ones: 0.53166026
ivories: 0.53024083
mannequins: 0.5294094
crypts: 0.5293703
gravestones: 0.52926946
p

keyboards: 0.5306373137088216
bass: 0.52891851186376
guitars: 0.5273030113199709
clarinet: 0.524479269981827
viols: 0.523160766376231
trumpet: 0.5218909163123633
drums: 0.521612518663866
bassoon: 0.5215483648381282
cellos: 0.5208607148167826
timpani: 0.5202116127783514
harpsichord: 0.5200590095835437
fiddle: 0.5199265225713982
piano: 0.5182590110326283
vocals: 0.5179630730709367
cello: 0.5175593389445461
saxes: 0.517130974655569
flute: 0.5162295723677758
guitar: 0.5158806854900627
flutes: 0.5156404209551363
clarinets: 0.515437464183464
below it the greenish water foamed over rocks and there  ___  lurking in the stony caves along the bank
producing top 20 simwords
rocks: 0.5249166
lies: 0.52484787
bolts: 0.5208078
snags: 0.5198816
sediment: 0.5195103
hides: 0.5182451
shells: 0.51716083
hangs: 0.5168102
anchors: 0.5158171
fossils: 0.5154061
skeletons: 0.5152232
remains: 0.51515824
icicles: 0.5148308
ledges: 0.5143902
seaweed: 0.5143696
urchins: 0.51427287
shimmers: 0.51394093
spines: 0.5

crepe: 0.5130861793145356
facial: 0.5130122118668847
black: 0.5123474151879606
napkin: 0.5115120931183679
bottle: 0.5114990211712506
drumstick: 0.5106217064383656
coat: 0.5103200339290704
cream: 0.5100080119510857
ginger: 0.5099601007792834
sack: 0.5098849651982793
skirt: 0.5098536895142628
trouser: 0.5093208908967201
tangerine: 0.5089446835571066
mustache: 0.508915676163978
cane: 0.5088980126038617
sock: 0.5088553317631731
ice-cream: 0.5087998747934029
cigar: 0.5087095806818898
towel: 0.5086277298143866
scotch: 0.5085599884860863
make a check list of essential items such as a  ___  bottle opener and a sharp knife for cutting bread etc
producing top 20 simwords
refill: 0.5723352
hot-water: 0.5481679
750ml: 0.5449793
foil: 0.5448633
plastic: 0.54376185
disposable: 0.54351526
lemonade: 0.5432628
mini: 0.54309577
refillable: 0.5427965
gas: 0.54071903
small: 0.5405572
jiffy: 0.5399035
homemade: 0.5394423
thermos: 0.53875107
glass: 0.5376981
metal: 0.53699553
home-made: 0.5368811
battery-op

bread: 0.5404906435664134
water: 0.5304355439845541
cider: 0.5287849362696158
corn: 0.5261783122265594
salt: 0.5243399154266087
wine: 0.5213570471044688
brandy: 0.5198909696693468
beer: 0.5193533332457819
coffee: 0.5189414920660742
flour: 0.5186882887971347
cheese: 0.5182467089439491
food: 0.5173310538224325
pudding: 0.5165421195701192
grain: 0.5164127350657144
meat: 0.5155412305004246
fish: 0.5154654652305379
sugar: 0.5150237446769694
milk: 0.514737031926574
lime: 0.514668968512011
mud: 0.5136271216490256
both  ___  and cattle were exported from britain under the romans which shows how successful their way of farming became
producing top 20 simwords
cattle: 0.5581931
sheep: 0.5535048
pigs: 0.55086094
horses: 0.5461105
deer: 0.5408416
beef: 0.5404697
oxen: 0.5403231
livestock: 0.54016984
farmers: 0.53912526
farms: 0.5388002
bulls: 0.5380363
hounds: 0.53789496
bullocks: 0.5361155
cows: 0.5348361
calves: 0.534492
beagles: 0.5340457
hogs: 0.5336956
chickens: 0.5327341
carcasses: 0.5326764